In [4]:
%pip install sentence-transformers chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import json

# 최종본 product_data.json 파일 경로
# .ipynb 파일과 같은 폴더에 있다면 파일 이름만 적으면 됩니다.
json_file_path = 'product_data.json' 

# 빈 리스트를 먼저 만듭니다.
product_documents = []

# json 파일을 열고 데이터를 읽어옵니다.
with open(json_file_path, 'r', encoding='utf-8') as f:
    products_data = json.load(f)

# for 루프를 사용해 각 제품 정보를 하나씩 꺼내옵니다.
for product in products_data:
    # plain text 형식으로 변환합니다. (이전과 동일한 형식)
    # f-string(f"...") 문법을 사용하면 편리합니다.
    text_block = (
        f"제품명: {product.get('제품명', '')}\n"
        f"브랜드명: {product.get('브랜드명', '')}\n"
        f"카테고리: {product.get('카테고리', '')}\n"
        f"효능: {product.get('효능', '').replace(';', ', ')}\n" # 세미콜론을 쉼표로 변경
        f"가격: {product.get('가격', 0):,}원\n" # 숫자에 콤마를 추가하고 '원'을 붙임
        f"용량: {product.get('용량', '')}\n"
        f"유해성 점수: {product.get('유해성_점수', '정보 없음')}\n"
        f"전성분: {product.get('전성분', '').replace(';', ', ')}\n" # 세미콜론을 쉼표로 변경
        f"링크: {product.get('링크', '')}"
    )
    
    # 완성된 텍스트 덩어리를 product_documents 리스트에 추가합니다.
    product_documents.append(text_block)

# 결과 확인: 리스트에 몇 개의 문서가 들어갔는지, 첫 번째 문서는 어떻게 생겼는지 출력
print(f"총 {len(product_documents)}개의 제품 문서가 생성되었습니다.")
print("\n--- 첫 번째 제품 예시 ---")
print(product_documents[0])

총 120개의 제품 문서가 생성되었습니다.

--- 첫 번째 제품 예시 ---
제품명: 아쿠아 오아시스 토너
브랜드명: 에스네이처
카테고리: 스킨/토너
효능: 보습, 진정
가격: 25,000원
용량: 300ml
유해성 점수: 1.090909090909091
전성분: 정제수, 다이프로필렌글라이콜, 부틸렌글라이콜, 글리세린, 1,2-헥산다이올, 베타인, 판테놀, 알란토인, 파파야열매추출물, 글루코노락톤, 자일리틸글루코사이드, 안하이드로자일리톨, 자일리톨, 소듐하이알루로네이트, 글루코오스, 파파인, 하이드록시프로필트라이모늄하이알루로네이트, 소듐아세틸레이티드하이알루로네이트, 하이드롤라이즈드하이알루로닉애씨드, 하이알루로닉애씨드, 하이드롤라이즈드소듐하이알루로네이트, 소듐하이알루로네이트크로스폴리머, 포타슘하이알루로네이트, 글리세릴아크릴레이트/아크릴릭애씨드코폴리머, 피브이엠/엠에이코폴리머
링크: https://www.hwahae.co.kr/goods/%EC%97%90%EC%8A%A4%EB%84%A4%EC%9D%B4%EC%B2%98-%EC%95%84%EC%BF%A0%EC%95%84-%EC%98%A4%EC%95%84%EC%8B%9C%EC%8A%A4-%ED%86%A0%EB%84%88-300ml-%EB%8D%94%EB%B8%94-%EA%B8%B0%ED%9A%8D-%ED%86%A0%EB%84%88PLUS%ED%86%A0%EB%84%88PLUS%ED%99%94%EC%9E%A5%EC%86%9C/26143?goods_tab=review_ingredients


In [6]:
# << 셀 1 >>

import json

json_file_path = 'product_data.json' 
product_documents = []

with open(json_file_path, 'r', encoding='utf-8') as f:
    products_data = json.load(f)

for product in products_data:
    text_block = (
        f"제품명: {product.get('제품명', '')}\n"
        f"브랜드명: {product.get('브랜드명', '')}\n"
        f"카테고리: {product.get('카테고리', '')}\n"
        f"효능: {product.get('효능', '').replace(';', ', ')}\n"
        f"가격: {product.get('가격', 0):,}원\n"
        f"용량: {product.get('용량', '')}\n"
        f"유해성 점수: {product.get('유해성_점수', '정보 없음')}\n"
        f"전성분: {product.get('전성분', '').replace(';', ', ')}\n"
        f"링크: {product.get('링크', '')}"
    )
    product_documents.append(text_block)

print(f"총 {len(product_documents)}개의 제품 문서가 생성되었습니다.")

총 120개의 제품 문서가 생성되었습니다.


In [7]:
# << 셀 2 >>

import chromadb
from sentence_transformers import SentenceTransformer

# 1단계에서 'product_documents' 리스트가 이미 만들어졌으므로, 여기서는 그냥 사용하면 됩니다.
# 복사/붙여넣기 할 필요가 전혀 없습니다!
# 궁금하다면 아래 코드의 주석(#)을 지우고 실행해서 개수를 직접 확인해봐도 좋습니다.
# print(f"메모리에서 가져온 문서 개수: {len(product_documents)}")

# 2. 한국어 임베딩 모델 불러오기
model = SentenceTransformer('jhgan/ko-sbert-nli')

# 3. 모든 제품 정보를 벡터로 변환 (임베딩 생성)
print("임베딩 생성을 시작합니다...")
embeddings = model.encode(product_documents)
print("임베딩 생성 완료!")

# 4. ChromaDB 클라이언트 설정 및 컬렉션(데이터 저장소) 생성
client = chromadb.Client()
collection = client.create_collection("products_db")

# 5. 벡터 데이터베이스에 데이터 저장
product_ids = [f"product_{i}" for i in range(len(product_documents))]

collection.add(
    embeddings=embeddings,
    documents=product_documents,
    ids=product_ids
)

print(f"\n총 {collection.count()}개의 제품 정보가 벡터 데이터베이스에 성공적으로 저장되었습니다.")

c:\Users\user\Desktop\sesac-pjt-final\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\Desktop\sesac-pjt-final\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--jhgan--ko-sbert-nli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to act

임베딩 생성을 시작합니다...
임베딩 생성 완료!

총 120개의 제품 정보가 벡터 데이터베이스에 성공적으로 저장되었습니다.


In [8]:
# 1. 사용자의 질문을 가정합니다.
query = "피부 진정에 좋은 토너 추천해줘"
# query = "가격이 저렴하면서 양 많은 로션 있어?" #<-- 다른 질문으로 바꿔서 테스트해보세요!

# 2. 질문도 제품 정보와 "동일한" 임베딩 모델을 사용해 벡터로 변환합니다.
query_embedding = model.encode(query)

# 3. DB에서 가장 유사한 N개의 문서를 검색합니다.
# n_results=3 은 가장 유사한 3개의 결과를 가져오라는 의미입니다.
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

# 4. 검색 결과를 확인합니다.
# 'documents' 안에 검색된 제품 정보(plain text)가 들어있습니다.
retrieved_documents = results['documents'][0]

print(f"--- '{query}'에 대한 검색 결과 ---")
for i, doc in enumerate(retrieved_documents):
    print(f"\n[결과 {i+1}]")
    print(doc)

--- '피부 진정에 좋은 토너 추천해줘'에 대한 검색 결과 ---

[결과 1]
제품명: 시카테롤 토너
브랜드명: 비플레인
카테고리: 스킨/토너
효능: 보습, 진정, 모공/피지
가격: 24,000원
용량: 200ml
유해성 점수: 1.25
전성분: 정제수, 프로판다이올, 병풀추출물, 1,2-헥산다이올, 베타-시토스테롤
링크: https://www.hwahae.co.kr/goods/%EB%B9%84%ED%94%8C%EB%A0%88%EC%9D%B8-%EC%8B%9C%EC%B9%B4%ED%85%8C%EB%A1%A4-%ED%86%A0%EB%84%88-200mlx2%EA%B0%9C/53048?goods_tab=review_ingredients

[결과 2]
제품명: 나인 토너
브랜드명: 닥터트웬티프로젝트
카테고리: 스킨/토너
효능: 보습, 진정, 모공/피지
가격: 29,000원
용량: 300ml
유해성 점수: 1.2
전성분: 정제수, 글리세린, 2,3-부탄다이올, 판테놀, 다이프로필렌글라이콜, 1,2-헥산다이올, 하이드록시아세토페논, 에리스리톨, 베타인, 트레할로오스, 알란토인, 에틸헥실글리세린, 소듐하이알루로네이트, 아크릴레이트/C10-30알킬아크릴레이트크로스폴리머, 알지닌, 다이소듐이디티에이, 쑥위추출물, 부틸렌글라이콜, 포타슘하이알루로네이트, 하이드롤라이즈드소듐하이알루로네이트, 로즈마리잎추출물, 자작나무수액, 트루로즈오브예리코추출물, 하이드롤라이즈드하이알루로닉애씨드, 하이알루로닉애씨드
링크: https://www.hwahae.co.kr/goods/%EB%8B%A5%ED%84%B0%ED%8A%B8%EC%9B%AC%ED%8B%B0%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EB%82%98%EC%9D%B8-%ED%86%A0%EB%84%88-300ml-1PLUS1/70460?goods_tab=review_ingredients

[결과 3]
제품명: 비피다 바이옴 앰플 토너
브랜드명: 마녀공장
카테고리: 스킨/토너
효능: 보습, 진정
가격: 25,

In [9]:
# 2단계에서 찾은 검색 결과(retrieved_documents)를 하나의 텍스트로 합칩니다.
context_for_llm = "\n\n".join(retrieved_documents)

# 이제 Claude에게 보낼 최종 프롬프트를 만듭니다.
final_prompt = f"""
너는 최고의 화장품 추천 전문가야. 아래 [참고 자료]를 바탕으로 사용자의 [질문]에 대해 친절하게 답변해줘.
반드시 [참고 자료]에 있는 내용만을 활용해서 답변해야 해.

[참고 자료]
{context_for_llm}

[질문]
{query}
"""

print("--- Claude에게 전달될 최종 프롬프트 ---")
print(final_prompt)

--- Claude에게 전달될 최종 프롬프트 ---

너는 최고의 화장품 추천 전문가야. 아래 [참고 자료]를 바탕으로 사용자의 [질문]에 대해 친절하게 답변해줘.
반드시 [참고 자료]에 있는 내용만을 활용해서 답변해야 해.

[참고 자료]
제품명: 시카테롤 토너
브랜드명: 비플레인
카테고리: 스킨/토너
효능: 보습, 진정, 모공/피지
가격: 24,000원
용량: 200ml
유해성 점수: 1.25
전성분: 정제수, 프로판다이올, 병풀추출물, 1,2-헥산다이올, 베타-시토스테롤
링크: https://www.hwahae.co.kr/goods/%EB%B9%84%ED%94%8C%EB%A0%88%EC%9D%B8-%EC%8B%9C%EC%B9%B4%ED%85%8C%EB%A1%A4-%ED%86%A0%EB%84%88-200mlx2%EA%B0%9C/53048?goods_tab=review_ingredients

제품명: 나인 토너
브랜드명: 닥터트웬티프로젝트
카테고리: 스킨/토너
효능: 보습, 진정, 모공/피지
가격: 29,000원
용량: 300ml
유해성 점수: 1.2
전성분: 정제수, 글리세린, 2,3-부탄다이올, 판테놀, 다이프로필렌글라이콜, 1,2-헥산다이올, 하이드록시아세토페논, 에리스리톨, 베타인, 트레할로오스, 알란토인, 에틸헥실글리세린, 소듐하이알루로네이트, 아크릴레이트/C10-30알킬아크릴레이트크로스폴리머, 알지닌, 다이소듐이디티에이, 쑥위추출물, 부틸렌글라이콜, 포타슘하이알루로네이트, 하이드롤라이즈드소듐하이알루로네이트, 로즈마리잎추출물, 자작나무수액, 트루로즈오브예리코추출물, 하이드롤라이즈드하이알루로닉애씨드, 하이알루로닉애씨드
링크: https://www.hwahae.co.kr/goods/%EB%8B%A5%ED%84%B0%ED%8A%B8%EC%9B%AC%ED%8B%B0%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EB%82%98%EC%9D%B8-%ED%86%A0%EB%84%88-300ml-1PLUS1/70460?goods_tab=revie

In [11]:
# << 새로운 코드 셀에 실행 >>
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# << 새로운 코드 셀에 실행 >>

import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
load_dotenv() 

# OPENAI_API_KEY 라는 이름의 키를 가져옵니다.
openai_api_key = os.getenv("OPENAI_API_KEY")

# 키가 잘 불러와졌는지 확인 (선택 사항)
if openai_api_key:
    print("OpenAI API 키를 성공적으로 불러왔습니다.")
else:
    print("OpenAI API 키를 불러오는 데 실패했습니다. .env 파일에 OPENAI_API_KEY='sk-...' 형식으로 저장되었는지 확인해주세요.")

OpenAI API 키를 성공적으로 불러왔습니다.


In [13]:
# << 새로운 코드 셀에 실행 >>

from openai import OpenAI

# 2단계에서 불러온 API 키로 OpenAI 클라이언트 초기화
client = OpenAI(api_key=openai_api_key)

# 이전에 검색 및 프롬프트 생성을 통해 만들어둔 final_prompt 변수를 그대로 사용합니다.
# 보내기 전에 내용을 다시 확인하고 싶다면 아래 코드의 주석을 해제하고 실행해보세요.
# print(final_prompt) 

print("\nOpenAI에 답변을 요청합니다...")

# OpenAI API 호출 (Chat Completion)
response = client.chat.completions.create(
  model="gpt-3.5-turbo",  # 저렴하고 빠른 gpt-3.5-turbo 모델을 사용합니다. "gpt-4" 등 다른 모델도 가능합니다.
  messages=[
    {"role": "user", "content": final_prompt}
  ]
)

# 답변 내용만 깔끔하게 추출해서 출력
chatbot_response = response.choices[0].message.content
print("\n--- 챗봇의 답변 ---")
print(chatbot_response)


OpenAI에 답변을 요청합니다...

--- 챗봇의 답변 ---
저희가 추천하는 피부 진정에 좋은 토너는 비피다 바이옴 앰플 토너입니다. 이 제품은 보습과 진정 효과가 있으며, 전성분에 포함된 락토바실러스발효용해물, 락토코쿠스발효용해물 등이 피부를 진정시켜줄 수 있습니다. 또한, 가격은 25,000원으로 가격대도 합리적입니다. 자세한 내용은 아래 링크를 참고해주세요. 
[비피다 바이옴 앰플 토너 링크] : https://www.hwahae.co.kr/goods/%EB%A7%88%EB%85%80%EA%B3%B5%EC%9E%A5-%EB%B9%84%ED%94%BC%EB%8B%A4-%EB%B0%94%EC%9D%B4%EC%98%B4-%EC%95%B0%ED%94%8C-%ED%86%A0%EB%84%88-400ml/35549?goods_tab=review_ingredients
